**Navigation** : [← Lean-8-Agentic-Proving](Lean-8-Agentic-Proving.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-10-LeanDojo →](Lean-10-LeanDojo.ipynb) --- # Lean 9 - Multi-Agents avec Semantic Kernel ## 1. Introduction Ce notebook explore l'utilisation de **Microsoft Semantic Kernel (SK)** pour orchestrer des agents autonomes dans la preuve de théorèmes Lean 4. **Prérequis** : Notebook Lean-8 (fondations agents ad-hoc, orchestration basique) **Objectifs** :
- Architecture multi-agents avec SK
- Plugins avec décorateur `@kernel_function`
- Coordination via `GroupChat` et stratégies de sélection/terminaison
- Gestion d'état partagé (`ProofState`)
- Intégration production **Durée estimée** : 50 min ---


## 🎯 Architecture du Système Multi-Agents ### 1.1. Vue d'ensemble Notre système utilise **5 agents spécialisés** qui collaborent pour prouver des théorèmes Lean : 1. **SearchAgent** : Recherche de lemmes pertinents dans Mathlib
2. **TacticAgent** : Génération de tactiques Lean appropriées
3. **VerifierAgent** : Vérification formelle des preuves
4. **CriticAgent** : Analyse et suggestions d'amélioration
5. **CoordinatorAgent** : Orchestration et décisions stratégiques ### 1.2. Pourquoi 5 agents ? Chaque agent a une **responsabilité unique** (principe de séparation des préoccupations) : - **Séparation des compétences** : Recherche ≠ Génération ≠ Vérification
- **Spécialisation** : Chaque LLM est prompté pour une tâche précise
- **Robustesse** : Si un agent échoue, les autres continuent
- **Traçabilité** : On sait quel agent a pris quelle décision ### 1.3. Communication : État partagé vs Message passing Deux approches classiques en multi-agents : | **Message Passing** | **État Partagé** (notre choix) |
|---------------------|--------------------------------|
| Agents s'envoient des messages | Tous les agents lisent/écrivent un état central |
| Décentralisé | Centralisé |
| Complexe à orchestrer | Facile à suivre |
| Pas de snapshot global | Snapshot complet à chaque itération | **Pourquoi état partagé ?** - Besoin de **cohérence globale** (historique des tactiques, métriques)
- **Debugging facilité** : On peut inspecter l'état après chaque tour
- **Snapshots JSON** : Permet de reproduire exactement une session
- Semantic Kernel supporte ce pattern avec les **plugins**

## 2. Integration avec Semantic Kernel (Python) ### 2.1. Vue d'ensemble Microsoft **Semantic Kernel** est un SDK qui permet d'orchestrer des LLMs avec des plugins, de la memoire et des agents intelligents. Nous allons implementer un systeme multi-agents pour theorem proving inspire des patterns utilises dans l'analyse argumentative (voir `Argument_Analysis` notebooks). **Composants cles** :
- **Kernel** : Point d'entree principal, configure les services LLM
- **Plugins** : Fonctions appelables par les agents (decorated avec `@kernel_function`)
- **Agents** : Entites autonomes avec instructions et capacites
- **Orchestration** : Strategies de selection et terminaison des agents ### 2.2. Dependances ```python
# Installation
pip install semantic-kernel openai python-dotenv
```

In [117]:
# =============================================================================
# Section 8.1 - ProofState: Etat Partage pour Multi-Agents
# =============================================================================
# Pattern inspire de RhetoricalAnalysisState dans Argument_Analysis
# Permet la synchronisation entre agents avec designation explicite

import os
import sys
import json
import time
import uuid
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any, Tuple
from datetime import datetime
from enum import Enum
from pathlib import Path

# --- Detection robuste du repertoire du notebook ---
# Fonctionne sous Windows, Linux, et WSL
notebook_dir = None

# Chemins connus (Windows et WSL)
KNOWN_PATHS = [
    Path("/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL
    Path("/mnt/c/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL (C:)
    Path("d:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),      # Windows
    Path("D:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),      # Windows
]

# Strategie 1: Variable d'environnement LEAN_NOTEBOOK_DIR
if os.getenv("LEAN_NOTEBOOK_DIR"):
    notebook_dir = Path(os.getenv("LEAN_NOTEBOOK_DIR"))
    if not (notebook_dir / "lean_runner.py").exists():
        notebook_dir = None

# Strategie 2: Chemins connus
if not notebook_dir:
    for known_path in KNOWN_PATHS:
        if known_path.exists() and (known_path / "lean_runner.py").exists():
            notebook_dir = known_path
            break

# Strategie 3: Chercher dans cwd et parents
if not notebook_dir:
    cwd = Path.cwd()
    candidates = [cwd, cwd / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"]

    # Remonter jusqu'a 5 niveaux
    current = cwd
    for _ in range(5):
        candidates.append(current)
        lean_path = current / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"
        if lean_path.exists():
            candidates.append(lean_path)
        if current.parent == current:
            break
        current = current.parent

    for candidate in candidates:
        if candidate.exists() and (candidate / "lean_runner.py").exists():
            notebook_dir = candidate
            break

# Strategie 4: Fallback sur cwd
if not notebook_dir:
    notebook_dir = Path.cwd()
    print(f"[WARN] lean_runner.py non trouve, fallback sur: {notebook_dir}")

# --- Charger .env ---
try:
    from dotenv import load_dotenv
    env_paths = [
        notebook_dir / ".env",
        notebook_dir.parent / ".env",
        Path.home() / ".env"
    ]
    for p in env_paths:
        if p.exists():
            load_dotenv(p, override=True)
            print(f"Configuration chargee depuis: {p}")
            break
    else:
        print("Aucun fichier .env trouve")
except ImportError:
    print("python-dotenv non installe")

# --- Importer lean_runner.py ---
if notebook_dir and str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

try:
    from lean_runner import LeanRunner, LeanResult
    print(f"lean_runner importe avec succes depuis {notebook_dir}")
except ImportError as e:
    print(f"ERREUR: Impossible d'importer lean_runner: {e}")
    print(f"Repertoire de travail: {Path.cwd()}")
    print(f"notebook_dir detecte: {notebook_dir}")
    print(f"sys.path: {sys.path[:5]}")
    raise

# --- Enumerations ---

class ProofStrategy(Enum):
    """Strategie de preuve en cours."""
    EXPLORATION = "exploration"      # Recherche initiale de lemmes
    REFINEMENT = "refinement"        # Affinage des tactiques
    VALIDATION = "validation"        # Verification finale
    RECOVERY = "recovery"            # Recuperation apres echecs

class TacticDifficulty(Enum):
    """Niveau de difficulte des tactiques."""
    SIMPLE = "simple"      # rfl, exact, omega
    INTERMEDIATE = "intermediate"  # simp, ring, linarith
    ADVANCED = "advanced"  # induction, cases

class ProofPhase(Enum):
    """Phase de la boucle de preuve."""
    INIT = "init"
    SEARCH = "search"
    GENERATE = "generate"
    VERIFY = "verify"
    ANALYZE = "analyze"
    COMPLETE = "complete"
    FAILED = "failed"

# --- ProofState: Etat partage entre agents ---

@dataclass
class TacticAttempt:
    """Une tentative de tactique."""
    tactic: str
    success: bool
    error: Optional[str] = None
    timestamp: datetime = field(default_factory=datetime.now)
    state_before: Optional[str] = None
    confidence: Optional[float] = None
    explanation: Optional[str] = None

@dataclass
class ProofState:
    """
    Etat partage entre les agents pour la preuve d'un theoreme.
    Permet la coordination sans couplage fort.
    """
    # Identifiants
    session_id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    theorem_name: str = ""
    theorem_statement: str = ""

    # Etat de la preuve
    current_goal: str = ""
    current_proof: List[str] = field(default_factory=list)
    phase: ProofPhase = ProofPhase.INIT
    strategy: ProofStrategy = ProofStrategy.EXPLORATION

    # Resultats des agents
    discovered_lemmas: List[str] = field(default_factory=list)
    generated_tactics: List[str] = field(default_factory=list)
    tactic_history: List[TacticAttempt] = field(default_factory=list)

    # Metriques
    iteration: int = 0
    max_iterations: int = 10
    start_time: datetime = field(default_factory=datetime.now)

    # Erreurs et diagnostics
    last_error: Optional[str] = None
    final_proof: Optional[str] = None
    error_count: int = 0

    # Verification tracking
    verification_results: List[Dict[str, Any]] = field(default_factory=list)
    total_lean_time_ms: float = 0.0

    # Agent designation for orchestration
    _next_agent: Optional[str] = field(default=None, repr=False)

    def add_tactic_attempt(self, tactic: str, state_before: Optional[str] = None,
                           confidence: Optional[float] = None, explanation: Optional[str] = None,
                           success: bool = False, error: Optional[str] = None) -> str:
        """Enregistre une tentative de tactique."""
        attempt_id = f"attempt_{len(self.tactic_history) + 1}"
        self.tactic_history.append(TacticAttempt(
            tactic=tactic,
            success=success,
            error=error,
            state_before=state_before,
            confidence=confidence,
            explanation=explanation
        ))
        if success:
            self.current_proof.append(tactic)
        else:
            self.error_count += 1
            self.last_error = error
        return attempt_id

    def add_lemma(self, name: str, statement: str, namespace: str = "", relevance: float = 0.5) -> str:
        """Ajoute un lemme decouvert a la liste."""
        lemma_id = f"{namespace}.{name}" if namespace else name
        lemma_info = f"{lemma_id}: {statement} (relevance: {relevance})"
        if lemma_info not in self.discovered_lemmas:
            self.discovered_lemmas.append(lemma_info)
        return lemma_id

    def get_context_summary(self) -> str:
        """Resume le contexte pour les agents."""
        return f"""
Theoreme: {self.theorem_name}
Enonce: {self.theorem_statement}
But actuel: {self.current_goal}
Phase: {self.phase.value}
Strategie: {self.strategy.value}
Iteration: {self.iteration}/{self.max_iterations}
Tactiques reussies: {len(self.current_proof)}
Erreurs: {self.error_count}
Derniere erreur: {self.last_error or 'Aucune'}
""".strip()


    

    # --- Properties for compatibility ---
    @property
    def tactics_history(self) -> List[TacticAttempt]:
        """Alias pour tactic_history (compatibilite)."""
        return self.tactic_history

    @property
    def proof_complete(self) -> bool:
        """True si la preuve est complete."""
        return self.phase == ProofPhase.COMPLETE
    
    @proof_complete.setter
    def proof_complete(self, value: bool):
        """Definit la completion de la preuve."""
        if value:
            self.phase = ProofPhase.COMPLETE
        elif self.phase == ProofPhase.COMPLETE:
            self.phase = ProofPhase.VERIFY
    
    @property
    def iteration_count(self) -> int:
        """Alias pour iteration (compatibilite)."""
        return self.iteration
    
    @iteration_count.setter
    def iteration_count(self, value: int):
        """Definit le compteur d'iterations."""
        self.iteration = value

    def increment_iteration(self):
        """Incremente le compteur d'iterations."""
        self.iteration += 1
    
    def designate_next_agent(self, agent_name: str):
        """Designe l'agent qui doit intervenir ensuite."""
        self._next_agent = agent_name
    
    def consume_next_agent_designation(self) -> Optional[str]:
        """Retourne et efface la designation d'agent."""
        agent = self._next_agent
        self._next_agent = None
        return agent
    
    def get_state_snapshot(self, summarize: bool = True) -> Dict[str, Any]:
        """Retourne un snapshot de l'etat pour les plugins."""
        if summarize:
            return {
                "session_id": self.session_id,
                "theorem": self.theorem_statement,
                "goal": self.current_goal,
                "phase": self.phase.value,
                "strategy": self.strategy.value,
                "iteration": f"{self.iteration}/{self.max_iterations}",
                "proof_steps": len(self.current_proof),
                "discovered_lemmas": len(self.discovered_lemmas),
                "errors": self.error_count,
                "last_error": self.last_error
            }
        else:
            return self.to_dict()


    def add_verification(self, attempt_id: str, success: bool, output: str, errors: str,
                         remaining_goals: Optional[str] = None, exec_time_ms: float = 0.0,
                         mode: str = "subprocess") -> str:
        """Enregistre un résultat de vérification Lean."""
        verif_id = f"verif_{len(self.verification_results) + 1}"
        self.verification_results.append({
            "id": verif_id,
            "attempt_id": attempt_id,
            "success": success,
            "output": output,
            "errors": errors,
            "remaining_goals": remaining_goals,
            "exec_time_ms": exec_time_ms,
            "mode": mode,
            "timestamp": datetime.now().isoformat()
        })
        return verif_id


    def set_proof_complete(self, proof: str):
        """Marque la preuve comme terminée et change la phase."""
        self.final_proof = proof
        self.phase = ProofPhase.COMPLETE


    def set_strategy(self, strategy: 'ProofStrategy'):
        """Change la stratégie de preuve."""
        self.strategy = strategy

    def to_dict(self) -> Dict[str, Any]:
        """Serialise l'etat."""
        return {
            "session_id": self.session_id,
            "theorem_name": self.theorem_name,
            "theorem_statement": self.theorem_statement,
            "current_goal": self.current_goal,
            "current_proof": self.current_proof,
            "phase": self.phase.value,
            "strategy": self.strategy.value,
            "discovered_lemmas": self.discovered_lemmas,
            "generated_tactics": self.generated_tactics,
            "iteration": self.iteration,
            "max_iterations": self.max_iterations,
            "error_count": self.error_count,
            "last_error": self.last_error
        }

# --- Test de l'initialisation ---
print("\n" + "="*60)
print("ProofState initialise avec succes")
print(f"LeanRunner disponible: {LeanRunner is not None}")
print("="*60)


Configuration chargee depuis: /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean/.env
lean_runner importe avec succes depuis /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean

ProofState initialise avec succes
LeanRunner disponible: True


### 2.3. Plugins Semantic Kernel L'architecture utilise 4 plugins specialises, chacun exposant des fonctions via `@kernel_function`: | Plugin | Role | Fonctions cles |
|--------|------|----------------|
| **ProofStateManagerPlugin** | Gestion de l'etat | get_proof_state, add_lemma, designate_next_agent |
| **LeanSearchPlugin** | Recherche Mathlib | search_mathlib_lemmas, check_lemma_type |
| **LeanTacticPlugin** | Generation tactiques | generate_tactics, analyze_tactic_failure |
| **LeanVerificationPlugin** | Verification Lean | verify_proof, verify_tactic_step | Ce pattern permet aux agents d'appeler ces fonctions automatiquement grace au `FunctionChoiceBehavior.Auto()` de Semantic Kernel.

## 🔌 Plugins Semantic Kernel : Exposer l'État aux Agents ### 2.4. Problème Les agents LLM ne peuvent pas accéder directement à `ProofState` (objet Python). ### 2.5. Solution : Plugins Un **plugin Semantic Kernel** expose des méthodes Python comme **fonctions appelables par le LLM**. ```python
@kernel_function( description="Enregistre une tentative de tactique", name="log_tactic_attempt"
)
def log_tactic_attempt(self, tactic: str, confidence: float) -> str: attempt_id = self._state.add_tactic_attempt(tactic, confidence=confidence) return f"Tactique {tactic} enregistrée avec ID {attempt_id}"
``` ### 2.6. Décorateur `@kernel_function` - `description` : Ce que le LLM voit ("À quoi sert cette fonction ?")
- `name` : Nom de la fonction pour le LLM
- Paramètres : Doivent correspondre **EXACTEMENT** à ce que le plugin appelle ### 2.7. Les 4 plugins 1. **log_tactic_attempt** : Enregistrer une tactique essayée
2. **add_verification_result** : Enregistrer le résultat Lean
3. **set_proof_strategy** : Changer la stratégie de recherche
4. **mark_proof_complete** : Déclarer la preuve terminée ### 2.8. Pourquoi c'est critique ? Sans plugins, le LLM ne peut que **parler** de preuves. Avec plugins, il peut **agir** : - Essayer des tactiques
- Vérifier formellement
- Ajuster sa stratégie en temps réel

In [118]:
# =============================================================================
# Section 8.2-8.5 - Plugins Semantic Kernel
# =============================================================================
# Architecture en 4 plugins specialises:
# - ProofStateManagerPlugin: Gestion de l'etat partage
# - LeanSearchPlugin: Recherche de lemmes Mathlib
# - LeanTacticPlugin: Generation de tactiques
# - LeanVerificationPlugin: Verification avec lean_runner.py

# Import du decorateur kernel_function
try:
    from semantic_kernel.functions import kernel_function
    SK_AVAILABLE = True
    print("Semantic Kernel disponible - utilisation des vrais decorateurs")
except ImportError:
    SK_AVAILABLE = False
    print("Semantic Kernel non disponible - mode simulation")
    # Decorateur de simulation
    def kernel_function(description="", name=None):
        def decorator(func):
            func._sk_function = True
            func._sk_description = description
            func._sk_name = name or func.__name__
            return func
        return decorator

# =============================================================================
# 8.2 ProofStateManagerPlugin
# =============================================================================

class ProofStateManagerPlugin:
    """
    Plugin pour gerer l'etat partage de la preuve.
    Expose les methodes de ProofState via @kernel_function.
    """

    def __init__(self, state: ProofState):
        self._state = state

    @kernel_function(
        description="Obtient un apercu de l'etat actuel de la preuve (theoreme, lemmes, tactiques, etc.)",
        name="get_proof_state"
    )
    def get_proof_state(self, summarize: bool = True) -> str:
        """Retourne l'etat actuel sous forme JSON."""
        snapshot = self._state.get_state_snapshot(summarize=summarize)
        return json.dumps(snapshot, indent=2, ensure_ascii=False)

    @kernel_function(
        description="Ajoute un lemme decouvert a l'etat partage",
        name="add_discovered_lemma"
    )
    def add_discovered_lemma(
        self, name: str, statement: str, namespace: str = "", relevance: float = 0.5
    ) -> str:
        """Enregistre un lemme trouve par SearchAgent."""
        lemma_id = self._state.add_lemma(name, statement, namespace, relevance)
        return f"Lemme ajoute: {lemma_id} ({name})"

    @kernel_function(
        description="Enregistre une tentative de tactique avec son niveau de confiance",
        name="log_tactic_attempt"
    )
    def log_tactic_attempt(
        self, tactic: str, state_before: str, confidence: float = 0.5, explanation: str = ""
    ) -> str:
        """Enregistre une tactique tentee par TacticAgent."""
        attempt_id = self._state.add_tactic_attempt(tactic, state_before, confidence, explanation)
        return f"Tactique enregistree: {attempt_id}"

    @kernel_function(
        description="Enregistre le resultat d'une verification Lean",
        name="add_verification_result"
    )
    def add_verification_result(
        self, attempt_id: str, success: bool, output: str, errors: str,
        remaining_goals: str = "", exec_time_ms: float = 0.0
    ) -> str:
        """Enregistre un resultat de verification."""
        verif_id = self._state.add_verification(
            attempt_id, success, output, errors,
            remaining_goals if remaining_goals else None, exec_time_ms, "subprocess"
        )
        status = "OK" if success else "ECHEC"
        return f"Verification {verif_id}: {status}"

    @kernel_function(
        description="Designe l'agent qui doit parler au prochain tour. IMPORTANT: utiliser le nom exact.",
        name="designate_next_agent"
    )
    def designate_next_agent(self, agent_name: str) -> str:
        """Delegue au prochain agent."""
        valid_agents = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]
        if agent_name not in valid_agents:
            return f"ERREUR: Agent invalide '{agent_name}'. Valides: {valid_agents}"
        self._state.designate_next_agent(agent_name)
        return f"Prochain agent: {agent_name}"

    @kernel_function(
        description="Marque la preuve comme terminee avec le code final",
        name="set_proof_complete"
    )
    def set_proof_complete(self, proof_code: str) -> str:
        """Marque la preuve comme reussie."""
        self._state.set_proof_complete(proof_code)
        return f"PREUVE COMPLETE! Code: {proof_code[:100]}..."

    @kernel_function(
        description="Change la strategie de preuve (exploration, refinement, validation, recovery)",
        name="set_proof_strategy"
    )
    def set_proof_strategy(self, strategy: str) -> str:
        """Change la strategie de preuve."""
        try:
            self._state.set_strategy(ProofStrategy(strategy))
            return f"Strategie changee: {strategy}"
        except ValueError:
            return f"ERREUR: Strategie invalide '{strategy}'. Valides: exploration, refinement, validation, recovery"


Semantic Kernel disponible - utilisation des vrais decorateurs


### 2.2. LeanSearchPlugin : Recherche de Lemmes Mathlib **Plugin exposant les méthodes de recherche** pour SearchAgent. #### 2.9.1. Méthodes Exposées ```python
@kernel_function
def search_lemmas(goal: str, keywords: List[str]) -> List[Lemma]: # Recherche dans Mathlib par keywords # Retourne lemmes triés par pertinence
``` **Pattern** : SearchAgent appelle ce plugin pour trouver lemmes Mathlib pertinents.


In [119]:


# =============================================================================
# 8.3 LeanSearchPlugin
# =============================================================================

class LeanSearchPlugin:
    """
    Plugin pour la recherche de lemmes dans Mathlib.
    Utilise des patterns connus + verification #check via lean_runner.
    """

    def __init__(self, runner: LeanRunner):
        self._runner = runner
        # Base de lemmes connus (extensible)
        self._known_lemmas = {
            # Arithmetique de base
            "Nat.add_zero": ("n + 0 = n", "Nat"),
            "Nat.zero_add": ("0 + n = n", "Nat"),
            "Nat.add_comm": ("n + m = m + n", "Nat"),
            "Nat.add_assoc": ("(n + m) + k = n + (m + k)", "Nat"),
            "Nat.mul_one": ("n * 1 = n", "Nat"),
            "Nat.one_mul": ("1 * n = n", "Nat"),
            "Nat.mul_comm": ("n * m = m * n", "Nat"),
            "Nat.mul_assoc": ("(n * m) * k = n * (m * k)", "Nat"),
            "Nat.left_distrib": ("n * (m + k) = n * m + n * k", "Nat"),
            "Nat.right_distrib": ("(n + m) * k = n * k + m * k", "Nat"),
            # Logique
            "And.intro": ("a -> b -> a /\\ b", "Logic"),
            "And.left": ("a /\\ b -> a", "Logic"),
            "And.right": ("a /\\ b -> b", "Logic"),
            "Or.inl": ("a -> a \\/ b", "Logic"),
            "Or.inr": ("b -> a \\/ b", "Logic"),
            "Eq.refl": ("a = a", "Logic"),
            "Eq.symm": ("a = b -> b = a", "Logic"),
            "Eq.trans": ("a = b -> b = c -> a = c", "Logic"),
        }

    @kernel_function(
        description="Recherche des lemmes Mathlib pertinents pour un but donne",
        name="search_mathlib_lemmas"
    )
    def search_mathlib_lemmas(self, goal: str, max_results: int = 10) -> str:
        """
        Recherche des lemmes par mots-cles.

        Args:
            goal: Description du but ou mots-cles (ex: "addition commutative")
            max_results: Nombre maximum de resultats

        Returns:
            JSON avec les lemmes trouves
        """
        goal_lower = goal.lower()
        results = []

        # Recherche par mots-cles
        keywords = goal_lower.replace("+", "add").replace("*", "mul").replace("=", "eq").split()

        for name, (statement, namespace) in self._known_lemmas.items():
            score = 0.0
            name_lower = name.lower()

            # Scoring par mots-cles
            for kw in keywords:
                if kw in name_lower:
                    score += 0.3
                if kw in statement.lower():
                    score += 0.2

            # Patterns specifiques
            if "comm" in goal_lower and "comm" in name_lower:
                score += 0.4
            if "assoc" in goal_lower and "assoc" in name_lower:
                score += 0.4
            if "zero" in goal_lower and "zero" in name_lower:
                score += 0.3
            if "distrib" in goal_lower and "distrib" in name_lower:
                score += 0.4

            if score > 0:
                results.append({
                    "name": name,
                    "statement": statement,
                    "namespace": namespace,
                    "relevance": min(score, 1.0)
                })

        # Trier par pertinence
        results.sort(key=lambda x: x["relevance"], reverse=True)
        return json.dumps(results[:max_results], indent=2, ensure_ascii=False)

    @kernel_function(
        description="Verifie qu'un lemme existe et retourne son type via #check",
        name="check_lemma_type"
    )
    def check_lemma_type(self, lemma_name: str) -> str:
        """
        Verifie l'existence d'un lemme via #check.

        Args:
            lemma_name: Nom du lemme (ex: "Nat.add_comm")

        Returns:
            JSON {exists, type, error}
        """
        code = f"#check {lemma_name}"
        result = self._runner.run(code)

        if result.success and not result.errors:
            # Extraire le type de la sortie
            return json.dumps({
                "exists": True,
                "type": result.output.strip(),
                "error": None
            })
        else:
            return json.dumps({
                "exists": False,
                "type": None,
                "error": result.errors or "Lemme non trouve"
            })


# =============================================================================
# 8.4 LeanTacticPlugin


### 2.3. Plugins de Tactiques et Verification Les deux plugins restants gerent la **generation de tactiques** et la **verification formelle** : #### 2.10.1. LeanTacticPlugin - **Responsabilite** : Generer des tactiques Lean adaptees au contexte
- **Methodes exposees** : - `generate_tactic()` : Genere une tactique basee sur goal + lemmes + historique - `estimate_confidence()` : Estime la probabilite de succes (0.0-1.0)
- **LLM-aware** : Utilise un prompt structure pour le LLM avec exemples de tactiques Lean
- **Strategies** : `exact`, `rw`, `apply`, `simp`, `induction`, `cases`, etc. #### 2.10.2. LeanVerificationPlugin - **Responsabilite** : Verifier les preuves via compilation Lean
- **Methodes exposees** : - `verify_proof()` : Compile le theoreme avec tactiques et retourne succes/echec - `parse_lean_errors()` : Parse les messages d'erreur Lean pour feedback agents
- **Detection de completion** : Reconnait "no goals" = preuve complete
- **Gestion d'erreurs** : Extrait type d'erreur (type mismatch, tactic failed, etc.) pour CriticAgent **Flow typique** :
```
SearchAgent trouve lemmes | v
TacticAgent genere tactique (via LeanTacticPlugin) | v
VerifierAgent compile (via LeanVerificationPlugin) | +-- Success → COMPLETE +-- Failure → CriticAgent analyse → retry
```


In [120]:
# =============================================================================

class LeanTacticPlugin:
    """
    Plugin pour la generation de tactiques.
    Fournit des heuristiques et analyse les echecs.
    """

    def __init__(self):
        # Tactiques par difficulte
        self._tactics = {
            "simple": ["rfl", "trivial", "exact ?_", "assumption"],
            "medium": ["simp", "omega", "decide", "constructor", "intro", "apply"],
            "complex": ["ring", "linarith", "aesop", "induction", "cases", "rcases"]
        }

        # Heuristiques par pattern de but
        self._heuristics = {
            "equality": ["rfl", "exact", "simp", "ring", "omega"],
            "forall": ["intro", "intros", "apply"],
            "exists": ["use", "exists", "exact"],
            "and": ["constructor", "exact And.intro"],
            "or": ["left", "right"],
            "implication": ["intro", "apply", "exact"],
            "nat_arithmetic": ["omega", "simp", "decide"],
            "ring_expression": ["ring", "ring_nf"]
        }

    @kernel_function(
        description="Genere des tactiques appropriees pour un but donne",
        name="generate_tactics"
    )
    def generate_tactics(self, goal: str, context: str = "", difficulty: str = "simple") -> str:
        """
        Genere des tactiques pour le but courant.

        Args:
            goal: Le but Lean a prouver
            context: Contexte additionnel (lemmes disponibles, etc.)
            difficulty: simple, medium, ou complex

        Returns:
            JSON [{tactic, confidence, explanation}]
        """
        suggestions = []
        goal_lower = goal.lower()

        # Detecter le type de but
        detected_patterns = []
        if "=" in goal:
            detected_patterns.append("equality")
        if "forall" in goal_lower or "∀" in goal:
            detected_patterns.append("forall")
        if "exists" in goal_lower or "∃" in goal:
            detected_patterns.append("exists")
        if "/\\" in goal or "∧" in goal or "And" in goal:
            detected_patterns.append("and")
        if "\\/" in goal or "∨" in goal or "Or" in goal:
            detected_patterns.append("or")
        if "->" in goal or "→" in goal:
            detected_patterns.append("implication")
        if any(x in goal_lower for x in ["nat", "n +", "m +", "+ 0", "0 +"]):
            detected_patterns.append("nat_arithmetic")
        if any(x in goal for x in ["*", "+"]) and "=" in goal:
            detected_patterns.append("ring_expression")

        # Collecter les tactiques suggeres
        seen = set()
        for pattern in detected_patterns:
            for tactic in self._heuristics.get(pattern, []):
                if tactic not in seen:
                    seen.add(tactic)
                    confidence = 0.7 if difficulty == "simple" else 0.5
                    suggestions.append({
                        "tactic": tactic,
                        "confidence": confidence,
                        "explanation": f"Pattern detecte: {pattern}"
                    })

        # Ajouter des tactiques de base
        base_tactics = self._tactics.get(difficulty, self._tactics["simple"])
        for tactic in base_tactics[:3]:
            if tactic not in seen:
                suggestions.append({
                    "tactic": tactic,
                    "confidence": 0.3,
                    "explanation": f"Tactique {difficulty} generique"
                })

        # Trier par confiance
        suggestions.sort(key=lambda x: x["confidence"], reverse=True)
        return json.dumps(suggestions[:8], indent=2, ensure_ascii=False)

    @kernel_function(
        description="Analyse un echec de tactique et suggere des alternatives",
        name="analyze_tactic_failure"
    )
    def analyze_tactic_failure(self, failed_tactic: str, error_msg: str) -> str:
        """
        Analyse pourquoi une tactique a echoue.

        Args:
            failed_tactic: La tactique qui a echoue
            error_msg: Message d'erreur Lean

        Returns:
            JSON {diagnosis, alternatives, error_type}
        """
        error_lower = error_msg.lower()
        diagnosis = ""
        alternatives = []
        error_type = "unknown"

        # Classifier l'erreur
        if "unknown identifier" in error_lower or "unknown constant" in error_lower:
            error_type = "unknown_identifier"
            diagnosis = "Lemme ou identifiant non reconnu. Verifier l'import ou le nom."
            alternatives = ["Chercher le bon nom avec #check", "Verifier les imports"]

        elif "type mismatch" in error_lower:
            error_type = "type_mismatch"
            diagnosis = "Les types ne correspondent pas. Verifier les arguments."
            alternatives = ["exact", "apply", "simp"]

        elif "unsolved goals" in error_lower or "goals remain" in error_lower:
            error_type = "unsolved_goals"
            diagnosis = "Des sous-buts restent. La tactique n'a pas complete la preuve."
            alternatives = ["Ajouter d'autres tactiques", "Essayer simp", "Decomposer avec have"]

        elif "tactic failed" in error_lower:
            error_type = "tactic_failed"
            diagnosis = f"La tactique '{failed_tactic}' n'a pas pu s'appliquer."
            # Suggerer des alternatives
            if failed_tactic in ["ring", "linarith"]:
                alternatives = ["omega", "simp", "decide"]
            elif failed_tactic == "simp":
                alternatives = ["simp only", "rfl", "exact"]
            else:
                alternatives = ["simp", "omega", "exact ?_"]

        elif "declaration uses 'sorry'" in error_lower:
            error_type = "sorry"
            diagnosis = "La preuve contient 'sorry' - incomplete."
            alternatives = ["Completer la preuve", "Remplacer sorry par une vraie tactique"]

        else:
            error_type = "other"
            diagnosis = f"Erreur non classifiee: {error_msg[:100]}"
            alternatives = ["Verifier la syntaxe", "Essayer une approche differente"]

        return json.dumps({
            "diagnosis": diagnosis,
            "alternatives": alternatives,
            "error_type": error_type,
            "original_error": error_msg[:200]
        }, indent=2, ensure_ascii=False)


### 2.4. LeanVerificationPlugin : Compilation et Vérification **Plugin exposant les méthodes de vérification** pour VerifierAgent. #### 2.11.1. Méthodes Exposées ```python
@kernel_function
def verify_proof(theorem: str, tactics: str) -> VerificationResult: # Compile le théorème avec tactiques # Parse output Lean (success/errors) # Détecte "no goals" = preuve complète
``` **Pattern** : VerifierAgent appelle ce plugin pour compiler preuves avec LeanRunner.


In [121]:


# =============================================================================
# 8.5 LeanVerificationPlugin
# =============================================================================

class LeanVerificationPlugin:
    """
    Plugin pour la verification des preuves avec lean_runner.
    """

    def __init__(self, runner: LeanRunner):
        self._runner = runner

    @kernel_function(
        description="Verifie une preuve complete (theoreme + tactiques)",
        name="verify_proof"
    )
    def verify_proof(self, theorem_statement: str, proof_tactics: str) -> str:
        """
        Verifie un theoreme avec sa preuve.

        Args:
            theorem_statement: L'enonce du theoreme (ex: "theorem add_zero (n : Nat) : n + 0 = n")
            proof_tactics: La preuve (ex: "exact Nat.add_zero n")

        Returns:
            JSON {success, output, errors, exec_time_ms, backend}
        """
        import time

        # Construire le code complet
        if "by" not in proof_tactics and ":=" not in proof_tactics:
            code = f"{theorem_statement} := by {proof_tactics}"
        elif ":=" in proof_tactics:
            code = f"{theorem_statement} {proof_tactics}"
        else:
            code = f"{theorem_statement} := {proof_tactics}"

        start = time.time()
        result = self._runner.run(code)
        exec_time = (time.time() - start) * 1000

        return json.dumps({
            "success": result.success,
            "output": result.output,
            "errors": result.errors,
            "exit_code": result.exit_code,
            "exec_time_ms": round(exec_time, 2),
            "backend": result.backend,
            "code": code
        }, indent=2, ensure_ascii=False)

    @kernel_function(
        description="Verifie une etape de tactique incrementale",
        name="verify_tactic_step"
    )
    def verify_tactic_step(
        self, partial_proof: str, next_tactic: str, theorem_statement: str
    ) -> str:
        """
        Verifie une tactique incrementale.

        Args:
            partial_proof: Les tactiques deja appliquees (separees par ;)
            next_tactic: La prochaine tactique a essayer
            theorem_statement: L'enonce du theoreme

        Returns:
            JSON {tactic_valid, remaining_goals, error, exec_time_ms}
        """
        import time

        # Combiner les tactiques
        if partial_proof:
            all_tactics = f"{partial_proof}; {next_tactic}"
        else:
            all_tactics = next_tactic

        code = f"{theorem_statement} := by {all_tactics}"

        start = time.time()
        result = self._runner.run(code)
        exec_time = (time.time() - start) * 1000

        # Analyser les goals restants
        remaining_goals = None
        if "unsolved goals" in result.errors.lower():
            # Extraire les goals du message d'erreur
            remaining_goals = result.errors

        return json.dumps({
            "tactic_valid": result.success or "unsolved goals" not in result.errors.lower(),
            "remaining_goals": remaining_goals,
            "error": result.errors if not result.success else None,
            "exec_time_ms": round(exec_time, 2),
            "applied_tactics": all_tactics
        }, indent=2, ensure_ascii=False)


# =============================================================================
# Test des Plugins
# =============================================================================

print("\n=== Test des Plugins ===")

# Creer l'etat et le runner
test_state = ProofState(theorem_statement="theorem test_add (n : Nat) : n + 0 = n")
runner = LeanRunner(backend="subprocess", timeout=30)

# Instancier les plugins
state_plugin = ProofStateManagerPlugin(test_state)
search_plugin = LeanSearchPlugin(runner)
tactic_plugin = LeanTacticPlugin()
verif_plugin = LeanVerificationPlugin(runner)

# Test 1: Recherche de lemmes
print("\n1. Recherche de lemmes pour 'addition zero':")
lemmas = search_plugin.search_mathlib_lemmas("addition zero", max_results=3)
print(lemmas)

# Test 2: Generation de tactiques
print("\n2. Tactiques pour 'n + 0 = n':")
tactics = tactic_plugin.generate_tactics("n + 0 = n", difficulty="simple")
print(tactics)

# Test 3: Verification avec lean_runner
print("\n3. Verification d'une preuve:")
result = verif_plugin.verify_proof("theorem test_rfl : 2 + 2 = 4", "rfl")
print(result)

# Test 4: Plugin StateManager
print("\n4. Ajout via StateManagerPlugin:")
print(state_plugin.add_discovered_lemma("Nat.add_zero", "n + 0 = n", "Nat", 0.9))
print(state_plugin.get_proof_state(summarize=True))



=== Test des Plugins ===

1. Recherche de lemmes pour 'addition zero':
[
  {
    "name": "Nat.add_zero",
    "statement": "n + 0 = n",
    "namespace": "Nat",
    "relevance": 0.6
  },
  {
    "name": "Nat.zero_add",
    "statement": "0 + n = n",
    "namespace": "Nat",
    "relevance": 0.6
  }
]

2. Tactiques pour 'n + 0 = n':
[
  {
    "tactic": "rfl",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "exact",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "simp",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "ring",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "omega",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "decide",
    "confidence": 0.7,
    "explanation": "Pattern detecte: nat_arithmetic"
  },
  {
    "tactic": "ring_nf",
    "confidence": 0.7,


## 🤖 Création des Agents Semantic Kernel ### 2.12. Anatomie d'un agent Chaque agent a : 1. **Un nom** : "SearchAgent", "TacticAgent", etc.
2. **Des instructions** : Prompt système qui définit son rôle
3. **Des plugins** : Fonctions qu'il peut appeler (via StatePlugin)
4. **Un modèle LLM** : GPT-5.2, Claude, etc. ### 2.13. Exemple : SearchAgent ```python
search_agent = kernel.add_agent( name="SearchAgent", instructions="""Tu es un expert en recherche de lemmes Mathlib. Ton rôle : Trouver les lemmes pertinents pour le but actuel. Délègue à TacticAgent une fois les lemmes trouvés.""", plugins=[state_plugin]
)
``` ### 2.14. Instructions : Le "métier" de l'agent Les instructions définissent : - **Responsabilité** : "Recherche de lemmes" vs "Génération de tactiques"
- **Critères de succès** : "Trouver au moins 2 lemmes pertinents"
- **Délégation** : "Quand déléguer à un autre agent ?" **Principe clé** : Instructions précises → Comportement prévisible ### 2.15. Pattern : Stratégies basées sur l'état Au lieu de coder en dur "SearchAgent → TacticAgent", on utilise : ```python
def select_next_agent(state: ProofState) -> str: if state.phase == ProofPhase.SEARCH: return "SearchAgent" elif state.phase == ProofPhase.TACTIC_GEN: return "TacticAgent" # ...
``` **Avantage** : Orchestration dynamique basée sur l'état réel de la preuve.

In [122]:
# =============================================================================
# Section 8.6 - Definition des 5 Agents Specialises avec Semantic Kernel
# =============================================================================
# Utilise ChatCompletionAgent de Semantic Kernel avec FunctionChoiceBehavior.Auto()

import os
import asyncio
from typing import Dict, Any, Optional

# --- Instructions des Agents ---

SEARCH_AGENT_INSTRUCTIONS = """
Tu es l'agent de RECHERCHE de lemmes pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Chercher des lemmes Mathlib pertinents pour le theoreme courant
- Identifier les lemmes qui peuvent aider a la preuve
- Enregistrer les lemmes trouves dans l'etat partage

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour comprendre le theoreme
2. Utilise search_mathlib_lemmas() avec des mots-cles pertinents
3. Verifie les lemmes prometteurs avec check_lemma_type()
4. Enregistre les lemmes utiles avec add_discovered_lemma()
5. Delegue a TacticAgent quand tu as trouve des lemmes

IMPORTANT:
- Cherche des lemmes LIES au but (egalites, arithmetique, logique)
- Delegation: Apres avoir trouve au moins 2-3 lemmes, delegue a TacticAgent
- Si aucun lemme pertinent, delegue quand meme a TacticAgent
"""

TACTIC_AGENT_INSTRUCTIONS = """
Tu es l'agent de GENERATION DE TACTIQUES pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Generer des sequences de tactiques Lean pour prouver le but
- Utiliser les lemmes trouves par SearchAgent
- Proposer des tactiques avec niveau de confiance

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir le theoreme et les lemmes
2. Utilise generate_tactics() pour obtenir des suggestions
3. Enregistre ta meilleure tentative avec log_tactic_attempt()
4. Delegue a VerifierAgent pour verification

STRATEGIES DE TACTIQUES (par difficulte):
- SIMPLE: rfl, trivial, exact, assumption
- MEDIUM: simp, omega, constructor, intro, apply
- COMPLEX: ring, linarith, induction, cases

IMPORTANT:
- Commence par les tactiques simples (rfl, exact)
- Utilise les lemmes trouves par SearchAgent (exact Nat.add_zero n)
- Delegation: TOUJOURS deleguer a VerifierAgent apres avoir propose
"""

VERIFIER_AGENT_INSTRUCTIONS = """
Tu es l'agent de VERIFICATION pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Verifier les tactiques proposees avec le compilateur Lean
- Enregistrer les resultats de verification
- Determiner si la preuve est complete ou s'il faut continuer

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir la derniere tactique
2. Utilise verify_proof() pour tester la preuve
3. Enregistre le resultat avec add_verification_result()
4. Si succes: set_proof_complete() et termine
5. Si echec: delegue a CriticAgent pour analyse

IMPORTANT:
- Teste TOUJOURS la derniere tactique proposee
- Si la preuve compile sans erreur, utilise set_proof_complete()
- Si echec, enregistre l'erreur et delegue a CriticAgent
"""

CRITIC_AGENT_INSTRUCTIONS = """
Tu es l'agent CRITIQUE pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Analyser les echecs de verification
- Diagnostiquer les erreurs Lean
- Orienter vers la bonne strategie de correction

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir les echecs recents
2. Utilise analyze_tactic_failure() pour comprendre l'erreur
3. Decide quelle direction prendre:
   - "unknown identifier" -> delegue a SearchAgent
   - "type mismatch" ou "tactic failed" -> delegue a TacticAgent
   - Echecs repetes (>3) -> delegue a CoordinatorAgent

IMPORTANT:
- Analyse les 3 derniers echecs pour detecter des patterns
- Si >3 echecs similaires, delegue a CoordinatorAgent
"""

COORDINATOR_AGENT_INSTRUCTIONS = """
Tu es l'agent COORDINATEUR (superviseur) pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Superviser l'ensemble de la session de preuve
- Debloquer les situations cycliques
- Ajuster la strategie globale

QUAND TU INTERVIENS:
- Appele par CriticAgent apres echecs repetes
- Appele si max_iterations approche
- Appele pour decisions strategiques majeures

IMPORTANT:
- Tu es le dernier recours, prends des decisions audacieuses
- Si >40 iterations, suggere de simplifier le theoreme
"""

# =============================================================================
# Detection de Semantic Kernel
# =============================================================================

SK_AVAILABLE = False
try:
    from semantic_kernel import Kernel
    from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
    from semantic_kernel.agents.strategies import (
        KernelFunctionSelectionStrategy,
        KernelFunctionTerminationStrategy,
    )
    from semantic_kernel.agents.strategies.selection.sequential_selection_strategy import SequentialSelectionStrategy
    from semantic_kernel.agents.strategies.termination.default_termination_strategy import DefaultTerminationStrategy
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
    from semantic_kernel.connectors.ai import FunctionChoiceBehavior
    from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
    from semantic_kernel.contents import ChatHistoryTruncationReducer
    from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
    from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
    from semantic_kernel.contents.chat_message_content import ChatMessageContent
    from pydantic import PrivateAttr
    SK_AVAILABLE = True
    print("Semantic Kernel disponible - utilisation de ChatCompletionAgent")
except ImportError as e:
    print(f"Semantic Kernel non disponible: {e}")
    print("Installation: pip install semantic-kernel")


Semantic Kernel disponible - utilisation de ChatCompletionAgent


### 2.5. SimpleAgent : Agent Fallback (Simulation) **Classe de secours** pour simuler agents quand Semantic Kernel non disponible. #### 2.16.1. Architecture ```python
class SimpleAgent: def __init__(self, name: str, instructions: str): self.name = name self.instructions = instructions def invoke(self, message: str) -> str: # Simulation simple basée sur règles # Utilisé en mode fallback si OpenAI API indisponible
``` **Usage** : Mode simulation pour tests sans LLM.


In [123]:

# =============================================================================
# Mode Simulation (fallback si SK non disponible)
# =============================================================================

class SimpleAgent:
    """
    Agent simplifie pour simulation ou fallback.
    Utilise directement l'API OpenAI si SK n'est pas disponible.
    """

    def __init__(
        self,
        name: str,
        instructions: str,
        plugins: Dict[str, Any],
        use_simulation: bool = True
    ):
        self.name = name
        self.instructions = instructions
        self.plugins = plugins
        self.use_simulation = use_simulation
        self._openai_client = None

        # Initialiser le client OpenAI si mode reel
        if not use_simulation:
            try:
                from openai import OpenAI
                api_key = os.getenv("OPENAI_API_KEY")
                if api_key and len(api_key) > 10 and not api_key.startswith("sk-..."):
                    self._openai_client = OpenAI(api_key=api_key)
            except ImportError:
                pass

    def _build_openai_tools(self) -> list:
        """Construit les outils au format OpenAI function calling."""
        import inspect
        tools = []
        for plugin_name, plugin in self.plugins.items():
            for attr_name in dir(plugin):
                attr = getattr(plugin, attr_name)
                if not callable(attr):
                    continue
                # Supporter les deux décorateurs
                is_sk_func = hasattr(attr, '_sk_function') or hasattr(attr, '__kernel_function__')
                if not is_sk_func:
                    continue

                sig = inspect.signature(attr)
                properties = {}
                required = []
                for param_name, param in sig.parameters.items():
                    if param_name == 'self':
                        continue
                    param_type = "string"
                    if param.annotation != inspect.Parameter.empty:
                        if param.annotation == bool:
                            param_type = "boolean"
                        elif param.annotation in (int, float):
                            param_type = "number"
                    properties[param_name] = {
                        "type": param_type,
                        "description": f"Parameter {param_name}"
                    }
                    if param.default == inspect.Parameter.empty:
                        required.append(param_name)

                # Obtenir nom et description
                if hasattr(attr, '__kernel_function_name__'):
                    func_name = attr.__kernel_function_name__
                    func_desc = getattr(attr, "__kernel_function_description__", "")
                elif hasattr(attr, '_sk_name'):
                    func_name = attr._sk_name
                    func_desc = getattr(attr, "_sk_description", "")
                else:
                    func_name = attr_name
                    func_desc = ""

                tools.append({
                    "type": "function",
                    "function": {
                        "name": f"{plugin_name}__{func_name}",
                        "description": func_desc,
                        "parameters": {
                            "type": "object",
                            "properties": properties,
                            "required": required
                        }
                    }
                })
        return tools

    def _execute_tool_call(self, tool_name: str, arguments: dict) -> str:
        """Execute un appel de fonction sur un plugin."""
        parts = tool_name.split("__", 1)
        if len(parts) != 2:
            return f"Erreur: format invalide: {tool_name}"

        plugin_name, func_name = parts
        plugin = self.plugins.get(plugin_name)
        if not plugin:
            return f"Erreur: plugin {plugin_name} non trouve"

        for attr_name in dir(plugin):
            attr = getattr(plugin, attr_name)
            if not callable(attr):
                continue
            is_sk = hasattr(attr, '_sk_function') or hasattr(attr, '__kernel_function__')
            if not is_sk:
                continue

            if hasattr(attr, '__kernel_function_name__'):
                name = attr.__kernel_function_name__
            elif hasattr(attr, '_sk_name'):
                name = attr._sk_name
            else:
                name = attr_name

            if name == func_name:
                try:
                    result = attr(**arguments)
                    return str(result)
                except Exception as e:
                    return f"Erreur {func_name}: {e}"

        return f"Erreur: {func_name} non trouve dans {plugin_name}"

    def invoke(self, message: str, state: ProofState) -> str:
        """Execute l'agent sur un message."""
        state.increment_iteration()

        if self.use_simulation or not self._openai_client:
            return self._simulate_response(message, state)
        else:
            return self._call_llm(message, state)

    def _simulate_response(self, message: str, state: ProofState) -> str:
        """Simulation de l'agent (sans appels LLM)."""
        if self.name == "SearchAgent":
            search = self.plugins.get("search")
            state_mgr = self.plugins.get("state")
            if search and state_mgr:
                lemmas_json = search.search_mathlib_lemmas(state.theorem_goal or "addition", max_results=3)
                lemmas = json.loads(lemmas_json)
                for lemma in lemmas[:2]:
                    state_mgr.add_discovered_lemma(lemma["name"], lemma["statement"], lemma["namespace"], lemma["relevance"])
                state_mgr.designate_next_agent("TacticAgent")
                return f"[SearchAgent] Trouves {len(lemmas[:2])} lemmes. Delegation a TacticAgent."

        elif self.name == "TacticAgent":
            tactic = self.plugins.get("tactic")
            state_mgr = self.plugins.get("state")
            if tactic and state_mgr:
                tactics_json = tactic.generate_tactics(state.theorem_goal or "n + 0 = n", difficulty="simple")
                tactics = json.loads(tactics_json)
                if tactics:
                    best = tactics[0]
                    state_mgr.log_tactic_attempt(best["tactic"], state.theorem_goal or "", best["confidence"], best["explanation"])
                state_mgr.designate_next_agent("VerifierAgent")
                return f"[TacticAgent] Propose: {tactics[0]['tactic'] if tactics else 'rfl'}. Delegation a VerifierAgent."

        elif self.name == "VerifierAgent":
            verif = self.plugins.get("verification")
            state_mgr = self.plugins.get("state")
            if verif and state_mgr and state.tactics_history:
                last_tactic = state.tactics_history[-1]
                result_json = verif.verify_proof(state.theorem_statement, last_tactic.tactic)
                result = json.loads(result_json)
                state_mgr.add_verification_result(
                    last_tactic.id, result["success"], result["output"], result["errors"],
                    "", result["exec_time_ms"]
                )
                if result["success"]:
                    state_mgr.set_proof_complete(last_tactic.tactic)
                    return f"[VerifierAgent] SUCCES! Preuve: {last_tactic.tactic}"
                else:
                    state_mgr.designate_next_agent("CriticAgent")
                    return f"[VerifierAgent] Echec: {result['errors'][:100]}. Delegation a CriticAgent."

        elif self.name == "CriticAgent":
            tactic = self.plugins.get("tactic")
            state_mgr = self.plugins.get("state")
            if tactic and state_mgr:
                failures = state.get_recent_failures(3)
                if failures:
                    _, last_verif = failures[0]
                    analysis_json = tactic.analyze_tactic_failure("unknown", last_verif.errors)
                    analysis = json.loads(analysis_json)
                    if analysis["error_type"] == "unknown_identifier":
                        state_mgr.designate_next_agent("SearchAgent")
                        return f"[CriticAgent] Identifiant inconnu. Retour a SearchAgent."
                    elif len(failures) > 3:
                        state_mgr.designate_next_agent("CoordinatorAgent")
                        return f"[CriticAgent] Trop d'echecs. Appel CoordinatorAgent."
                    else:
                        state_mgr.designate_next_agent("TacticAgent")
                        return f"[CriticAgent] Essayer d'autres tactiques. -> TacticAgent."
                state_mgr.designate_next_agent("TacticAgent")
                return "[CriticAgent] Pas d'echecs recents. -> TacticAgent."

        elif self.name == "CoordinatorAgent":
            state_mgr = self.plugins.get("state")
            if state_mgr:
                if state.iteration_count > 30:
                    state_mgr.set_proof_strategy("recovery")
                    return "[CoordinatorAgent] Mode recovery active."
                else:
                    state_mgr.set_proof_strategy("refinement")
                    state_mgr.designate_next_agent("TacticAgent")
                    return "[CoordinatorAgent] Strategie refinement. -> TacticAgent."

        return f"[{self.name}] Action simulee."

    def _call_llm(self, message: str, state: ProofState) -> str:
        """Appelle le LLM OpenAI avec function calling."""
        state_summary = json.dumps(state.get_state_snapshot(summarize=True), indent=2)
        tools = self._build_openai_tools()

        nl = chr(10)
        user_content = f"ETAT ACTUEL:{nl}{state_summary}{nl}{nl}TACHE:{nl}{message}"
        messages = [
            {"role": "system", "content": self.instructions},
            {"role": "user", "content": user_content}
        ]

        max_tool_calls = 10
        tool_results = []

        for iteration in range(max_tool_calls):
            try:
                model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")
                use_mct = any(model.startswith(p) for p in ('gpt-4.5', 'gpt-5', 'o1', 'o3'))
                token_param = {"max_completion_tokens": 1000} if use_mct else {"max_tokens": 1000}

                response = self._openai_client.chat.completions.create(
                    model=model,
                    messages=messages,
                    tools=tools if tools else None,
                    tool_choice="auto" if tools else None,
                    temperature=0.3,
                    **token_param
                )

                assistant_message = response.choices[0].message

                if assistant_message.tool_calls:
                    messages.append(assistant_message.model_dump())

                    for tool_call in assistant_message.tool_calls:
                        func_name = tool_call.function.name
                        try:
                            arguments = json.loads(tool_call.function.arguments)
                        except json.JSONDecodeError:
                            arguments = {}

                        result = self._execute_tool_call(func_name, arguments)
                        tool_results.append(func_name.split("__")[-1])

                        messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": result
                        })
                else:
                    final_response = assistant_message.content or "(pas de reponse)"
                    if tool_results:
                        actions = ", ".join(tool_results[:5])
                        final_response = f"Actions: {actions}{nl}{final_response}"
                    return f"[{self.name}] {final_response}"

            except Exception as e:
                return f"[{self.name}] Erreur LLM: {e}"

        actions = ", ".join(tool_results[:5])
        return f"[{self.name}] Max tool calls. Actions: {actions}"


In [124]:


# =============================================================================
# Factory pour creer les agents (SK ou fallback)
# =============================================================================

def create_agents(
    plugins: Dict[str, Any],
    state: ProofState,
    use_sk: bool = True,
    use_simulation: bool = False
) -> Dict[str, Any]:
    """
    Cree les 5 agents specialises.

    Args:
        plugins: Dictionnaire des plugins SK
        state: Etat partage de la preuve
        use_sk: Utiliser Semantic Kernel si disponible
        use_simulation: Mode simulation (sans appels LLM)

    Returns:
        Dictionnaire {nom_agent: agent}
    """
    if use_sk and SK_AVAILABLE and not use_simulation:
        return _create_sk_agents(plugins, state)
    else:
        return _create_simple_agents(plugins, use_simulation)


def _create_simple_agents(plugins: Dict[str, Any], use_simulation: bool) -> Dict[str, Any]:
    """Cree les agents en mode fallback/simulation."""
    return {
        "SearchAgent": SimpleAgent("SearchAgent", SEARCH_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "TacticAgent": SimpleAgent("TacticAgent", TACTIC_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "VerifierAgent": SimpleAgent("VerifierAgent", VERIFIER_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "CriticAgent": SimpleAgent("CriticAgent", CRITIC_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "CoordinatorAgent": SimpleAgent("CoordinatorAgent", COORDINATOR_AGENT_INSTRUCTIONS, plugins, use_simulation),
    }


def _create_sk_agents(plugins: Dict[str, Any], state: ProofState) -> Dict[str, Any]:
    """
    Cree les agents avec Semantic Kernel ChatCompletionAgent.

    Utilise:
    - OpenAIChatCompletion pour le service LLM
    - FunctionChoiceBehavior.Auto() pour le function calling automatique
    - Les plugins existants sont passes aux agents
    """
    # Creer le kernel et le service
    kernel = Kernel()
    api_key = os.getenv("OPENAI_API_KEY")
    model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")

    service = OpenAIChatCompletion(
        service_id="openai",
        ai_model_id=model,
        api_key=api_key
    )
    kernel.add_service(service)

    # Ajouter les plugins au kernel
    for plugin_name, plugin in plugins.items():
        kernel.add_plugin(plugin, plugin_name=plugin_name)

    # Configuration pour auto function calling
    settings = kernel.get_prompt_execution_settings_from_service_id(service_id="openai")
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Creer les agents
    agents = {}
    agent_configs = [
        ("SearchAgent", SEARCH_AGENT_INSTRUCTIONS),
        ("TacticAgent", TACTIC_AGENT_INSTRUCTIONS),
        ("VerifierAgent", VERIFIER_AGENT_INSTRUCTIONS),
        ("CriticAgent", CRITIC_AGENT_INSTRUCTIONS),
        ("CoordinatorAgent", COORDINATOR_AGENT_INSTRUCTIONS),
    ]

    for name, instructions in agent_configs:
        agents[name] = ChatCompletionAgent(
            kernel=kernel,
            name=name,
            instructions=instructions,
            arguments=KernelArguments(settings=settings)
        )

    print(f"Crees {len(agents)} agents SK avec modele {model}")
    return agents


# =============================================================================
# Test des Agents
# =============================================================================

print("\n=== Test des Agents ===")

# Creer l'environnement
test_state = ProofState(
    theorem_statement="theorem add_zero (n : Nat) : n + 0 = n",
    current_goal="n + 0 = n"
)
runner = LeanRunner(backend="subprocess", timeout=30)

# Creer les plugins
plugins = {
    "state": ProofStateManagerPlugin(test_state),
    "search": LeanSearchPlugin(runner),
    "tactic": LeanTacticPlugin(),
    "verification": LeanVerificationPlugin(runner)
}

# Mode de fonctionnement
USE_SK = SK_AVAILABLE and os.getenv("OPENAI_API_KEY")
USE_SIMULATION = not USE_SK  # Simulation si SK non disponible ou pas de cle API

print(f"Mode: {'Semantic Kernel' if USE_SK else 'Simulation'}")

# Creer les agents
agents = create_agents(plugins, test_state, use_sk=USE_SK, use_simulation=USE_SIMULATION)

# Test rapide en mode simulation
if USE_SIMULATION:
    print("\nTest SearchAgent (simulation):")
    response = agents["SearchAgent"].invoke("Trouve des lemmes pour n + 0 = n", test_state)
    print(response)
    print(f"Etat apres SearchAgent:\n{test_state}")



=== Test des Agents ===
Mode: Semantic Kernel
Crees 5 agents SK avec modele gpt-5.2


### 2.6. ProofSelectionStrategy : Selection d'Agent Basee sur l'Etat **Pattern inspire de Argument_Analysis** : Selection d'agent via **designation explicite** dans ProofState. #### 2.17.1. Architecture ```python
class ProofSelectionStrategy(SelectionStrategy): async def next(agents, history) -> Agent: # 1. Lire designation explicite designated = state.consume_next_agent_designation() # 2. Si designation presente, utiliser cet agent if designated: return agents_map[designated] # 3. Sinon, utiliser agent par defaut (SearchAgent) return agents_map[default_agent_name]
``` #### 2.17.2. Difference avec Semantic Kernel Standard | Semantic Kernel Standard | ProofSelectionStrategy (Custom) |
|-------------------------|--------------------------------|
| Parse historique messages | Lit `state.next_agent_designation` |
| Selection basee sur keywords | Selection basee sur etat partage |
| Stateless (pas de memoire) | Stateful (ProofState) |
| Complexite O(n) messages | Complexite O(1) | **Avantage** : Chaque agent designe explicitement son successeur via `state.designate_next_agent()`, evitant parsing d'historique fragile.


In [126]:

# =============================================================================
# ProofSelectionStrategy - Selection basee sur l'etat partage
# =============================================================================

class ProofSelectionStrategy(SelectionStrategy):
    """
    Strategie de selection d'agent basee sur l'etat partage ProofState.
    Lit state.consume_next_agent_designation() au lieu de parser l'historique.
    Pattern inspire de Argument_Analysis/DelegatingSelectionStrategy.
    """

    _agents_map: Dict[str, Any] = PrivateAttr()
    _state: 'ProofState' = PrivateAttr()
    _default_agent_name: str = PrivateAttr()
    _logger: logging.Logger = PrivateAttr()

    def __init__(self, agents: List[Any], state: 'ProofState',
                 default_agent_name: str = "SearchAgent"):
        super().__init__()
        if not hasattr(state, 'consume_next_agent_designation'):
            raise TypeError("state doit avoir consume_next_agent_designation()")
        self._agents_map = {agent.name: agent for agent in agents}
        self._state = state
        self._default_agent_name = default_agent_name
        self._logger = logging.getLogger("ProofSelectionStrategy")

    async def next(self, agents: List[Any], history: List[ChatMessageContent]) -> Any:
        """Selectionne le prochain agent base sur l'etat partage."""
        # 1. Verifier designation explicite dans l'etat
        try:
            designated = self._state.consume_next_agent_designation()
            if designated:
                self._logger.info(f"Designation explicite: {designated}")
                if designated in self._agents_map:
                    return self._agents_map[designated]
                self._logger.warning(f"Agent '{designated}' non trouve!")
        except Exception as e:
            self._logger.error(f"Erreur lecture designation: {e}")

        # 2. Logique basee sur la phase
        try:
            phase = self._state.phase.value
            if phase == "search":
                return self._agents_map.get("TacticAgent", self._agents_map.get(self._default_agent_name))
            elif phase == "generate":
                return self._agents_map.get("VerifierAgent", self._agents_map.get(self._default_agent_name))
            elif phase == "verify":
                if self._state.last_error:
                    return self._agents_map.get("CriticAgent", self._agents_map.get(self._default_agent_name))
                return self._agents_map.get("CoordinatorAgent", self._agents_map.get(self._default_agent_name))
            elif phase == "analyze":
                return self._agents_map.get("SearchAgent", self._agents_map.get(self._default_agent_name))
        except Exception as e:
            self._logger.error(f"Erreur logique phase: {e}")

        return self._agents_map.get(self._default_agent_name)

    async def reset(self) -> None:
        try:
            self._state.consume_next_agent_designation()
        except:
            pass


In [127]:


# =============================================================================
# ProofTerminationStrategy - Terminaison basee sur l'etat partage
# =============================================================================

class ProofTerminationStrategy(TerminationStrategy):
    """
    Strategie de terminaison basee sur l'etat partage ProofState.
    Verifie state.proof_complete au lieu d'analyser les messages.
    """

    _state: 'ProofState' = PrivateAttr()
    _max_iterations: int = PrivateAttr()
    _logger: logging.Logger = PrivateAttr()

    def __init__(self, state: 'ProofState', max_iterations: int = 15):
        super().__init__()
        if not hasattr(state, 'proof_complete'):
            raise TypeError("state doit avoir proof_complete")
        self._state = state
        self._max_iterations = max_iterations
        self._logger = logging.getLogger("ProofTerminationStrategy")

    async def should_terminate(self, agent: Any, history: List[ChatMessageContent]) -> bool:
        """Verifie si la conversation doit se terminer."""
        # 1. Preuve trouvee
        try:
            if self._state.proof_complete:
                self._logger.info("Terminaison: preuve complete")
                return True
        except Exception as e:
            self._logger.error(f"Erreur verification preuve: {e}")

        # 2. Max iterations
        try:
            if self._state.iteration >= self._max_iterations:
                self._logger.info(f"Terminaison: max iterations ({self._max_iterations})")
                return True
        except Exception as e:
            self._logger.error(f"Erreur verification iterations: {e}")

        # 3. Phase complete ou failed
        try:
            if self._state.phase.value in ["complete", "failed"]:
                self._logger.info(f"Terminaison: phase {self._state.phase.value}")
                return True
        except:
            pass

        return False

    async def reset(self) -> None:
        pass


# =============================================================================
# ProofAgentGroupChat - Orchestration multi-agents
# =============================================================================



### 2.7. ProofAgentGroupChat : Orchestration Multi-Agents **Classe orchestrateur** pour gérer la conversation multi-agents avec Semantic Kernel. #### 2.18.1. Architecture ```python
class ProofAgentGroupChat: def __init__(agents, state, use_sk=True): self.agents = agents # Dict[str, ChatCompletionAgent] self.state = state # ProofState partage def run(initial_message, verbose=True) -> str: # Execute conversation multi-agents if use_sk: return await _run_sk(...) # Semantic Kernel else: return _run_fallback(...) # Simulation
``` **Pattern clé** :
- Utilise `ProofSelectionStrategy` pour sélectionner agents
- Utilise `ProofTerminationStrategy` pour détecter fin
- Crée `AgentGroupChat` de Semantic Kernel avec ces stratégies
- Fallback en mode simulation si SK non disponible


In [128]:
class ProofAgentGroupChat:
    """
    Orchestre les agents pour la preuve de theoremes.
    Utilise ProofSelectionStrategy et ProofTerminationStrategy.
    """

    def __init__(self, agents: Dict[str, Any], state: ProofState, use_sk: bool = True):
        self.agents = agents
        self.state = state
        self.use_sk = use_sk and SK_AVAILABLE
        self.history = []

    def run(self, initial_message: str, verbose: bool = True) -> str:
        """Execute la conversation multi-agents."""
        if self.use_sk:
            return asyncio.get_event_loop().run_until_complete(
                self._run_sk(initial_message, verbose)
            )
        else:
            return self._run_fallback(initial_message, verbose)

    async def _run_sk(self, initial_message: str, verbose: bool) -> str:
        """Execution avec Semantic Kernel et strategies personnalisees."""
        if verbose:
            print("=" * 60)
            print(f"Session SK demarree: {initial_message[:80]}...")
            print("=" * 60)

        agent_list = list(self.agents.values())

        if verbose:
            print(f"[LOG] Agents: {[a.name for a in agent_list]}")
            print(f"[LOG] Max iterations: {self.state.max_iterations}")

        # Creer les strategies basees sur l'etat partage
        selection_strategy = ProofSelectionStrategy(
            agents=agent_list,
            state=self.state,
            default_agent_name="SearchAgent"
        )

        termination_strategy = ProofTerminationStrategy(
            state=self.state,
            max_iterations=self.state.max_iterations
        )

        if verbose:
            print(f"[LOG] Strategies initialisees (basees sur etat partage)")

        # Creer le chat SK
        chat = AgentGroupChat(
            agents=agent_list,
            selection_strategy=selection_strategy,
            termination_strategy=termination_strategy,
        )

        await chat.add_chat_message(message=initial_message)

        iteration = 0
        if verbose:
            print(f"[LOG] Demarrage boucle multi-agents...")

        try:
            async for response in chat.invoke():
                iteration += 1
                self.state.iteration = iteration

                if response is None:
                    if verbose:
                        print(f"[Tour {iteration}] Response None, continue...")
                    continue

                agent_name = getattr(response, 'name', None) or 'Unknown'
                content = str(response.content)[:500] if response.content else "(vide)"

                if verbose:
                    print(f"\n[Tour {iteration}/{self.state.max_iterations}] Agent: {agent_name}")
                    print(f"  Phase: {self.state.phase.value}")
                    print(f"  Response: {content[:300]}{'...' if len(content) > 300 else ''}")

                self.history.append({
                    "agent": agent_name,
                    "response": content,
                    "iteration": iteration,
                    "phase": self.state.phase.value,
                })

                if self.state.proof_complete:
                    if verbose:
                        print(f"[LOG] Preuve complete detectee!")
                    break

        except Exception as e:
            if verbose:
                print(f"\n[ERREUR Tour {iteration}] {type(e).__name__}: {e}")
                import traceback
                traceback.print_exc()

        if verbose:
            print("\n" + "=" * 60)
            print(f"SESSION TERMINEE - {iteration} tours executes")
            if self.state.proof_complete:
                print("SUCCES! Preuve trouvee:")
                print(self.state.final_proof)
            else:
                print(f"Pas de preuve trouvee.")
                if self.state.last_error:
                    print(f"Derniere erreur: {self.state.last_error}")
            print("=" * 60)

        return self.state.final_proof or "Preuve non trouvee"

    def _run_fallback(self, initial_message: str, verbose: bool = True) -> str:
        """Execution sans Semantic Kernel (mode simulation)."""
        if verbose:
            print("=" * 60)
            print(f"Session FALLBACK: {initial_message[:80]}...")
            print("=" * 60)

        current_message = initial_message
        agent_order = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]

        for i in range(self.state.max_iterations):
            designated = self.state.consume_next_agent_designation()
            if designated and designated in self.agents:
                agent_name = designated
            else:
                agent_name = agent_order[i % len(agent_order)]

            agent = self.agents.get(agent_name)
            if not agent:
                continue

            if verbose:
                print(f"\n[Tour {self.state.iteration_count + 1}] {agent_name}")

            response = agent.invoke(current_message, self.state)

            self.history.append({
                "iteration": self.state.iteration_count,
                "agent": agent_name,
                "response": response[:200],
            })

            if verbose:
                print(f"  Response: {response[:200]}...")

            if self.state.proof_complete:
                if verbose:
                    print(f"\n[LOG] Preuve trouvee!")
                break

            current_message = response

        if verbose:
            print("\n" + "=" * 60)
            print(f"Session terminee apres {self.state.iteration_count} iterations.")
            print("=" * 60)

        return self.state.final_proof or "Preuve non trouvee"


# =============================================================================


In [129]:
# Test des Strategies
# =============================================================================

print("=== Test des Strategies ===")

test_state = ProofState(
    theorem_statement="theorem test (n : Nat) : n = n",
    current_goal="n = n",
    max_iterations=5
)

print(f"State cree: {test_state.session_id}")
print(f"Phase initiale: {test_state.phase.value}")

# Test designation
test_state.designate_next_agent("TacticAgent")
designated = test_state.consume_next_agent_designation()
print(f"Designation test: {designated}")

# Test proof_complete
print(f"proof_complete initial: {test_state.proof_complete}")
test_state.phase = ProofPhase.COMPLETE
print(f"proof_complete apres COMPLETE: {test_state.proof_complete}")

print("\nStrategies pretes pour utilisation avec AgentGroupChat")


=== Test des Strategies ===
State cree: 9fe1c266
Phase initiale: init
Designation test: TacticAgent
proof_complete initial: False
proof_complete apres COMPLETE: True

Strategies pretes pour utilisation avec AgentGroupChat



--- ## 3. Conclusion Vous avez maintenant une architecture complète de multi-agents avec Semantic Kernel pour la preuve automatique de théorèmes Lean 4. **Prochain notebook** : Lean-10-LeanDojo (applications avancées) **Ressources** :
- [Semantic Kernel Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Lean 4 Documentation](https://lean-lang.org/documentation/)
- [Mathlib4](https://github.com/leanprover-community/mathlib4)
